# Projet Data Mining : Les crimes à San Franscico | Exploration des données

Notre groupe (Emeline Pinquier, Jean-Pierre Mohamed et Aurélien Khefif-Derain) a décidé de travailler sur la compétition suivante : https://www.kaggle.com/c/sf-crime
Le but est de trouver le type de crime grâce à : 
    - Date (YYYY-MM-DD HH:mm:SS)
    - Jour de la semaine ("Monday","Tuesday","Wednesday","Thursday"....)
    - Quartier ("BAYVIEW","NORTHERN"...)
    - Adresse ("2000 Block of THOMAS AV","3RD ST / REVERE AV"...)
    - Coordonnée X
    - Coordonnée Y
    
La catégorie du crime peut prendre la forme suivante :
    - WARRANTS
    - VANDALISM
    - ASSAULT
    - FRAUD
    ...
    


## Import des modules

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## Chargement du jeu de données

### Chargement de train.csv 

In [ ]:
data = pd.read_csv('../input/train.csv', parse_dates=['Dates'])
print(data.shape)
data.head(3)


### Chargement de test.csv

In [ ]:
test = pd.read_csv('../input/test.csv', parse_dates=['Dates'])
print(test.shape)
test.head(3)

## Exploration des données

Nous allons vérifier les champs seulement les champs qui sont disponible dans le test.csv (c'est le jeu de test que nous devons en suite envoyer à Kaggle). Donc tout les champs seront étudiés à l'exception de la description et la catégorie.

In [ ]:
data.info()

### 1. Dates 

Nous allons diviser les dates en Année "Year", Mois "Month", Jour "Day", Heure "Hour", Minute "Minute" et Seconde "Second"

In [ ]:
data['Dates-year'] = data['Dates'].dt.year
data['Dates-month'] = data['Dates'].dt.month
data['Dates-day'] = data['Dates'].dt.day
data['Dates-hour'] = data['Dates'].dt.hour
data['Dates-minute'] = data['Dates'].dt.minute
data['Dates-second'] = data['Dates'].dt.second

In [ ]:
fig, ((axis1,axis2,axis3),(axis4,axis5,axis6)) = plt.subplots(nrows=2, ncols=3)
fig.set_size_inches(18,6)

sns.countplot(data=data, x='Dates-year', ax=axis1)
sns.countplot(data=data, x='Dates-month', ax=axis2)
sns.countplot(data=data, x='Dates-day', ax=axis3)
sns.countplot(data=data, x='Dates-hour', ax=axis4)
sns.countplot(data=data, x='Dates-minute', ax=axis5)
sns.countplot(data=data, x='Dates-second', ax=axis6)

Nous pouvons déjà tirer plusieurs conclusions.
    - Les **secondes** sont inutiles car par défaut elles sont toutes à 0
    - Les **jours** : Il y en a moins le 31, car c'est un jour qui n'apparait pas tout les mois..
    - Les **heures** : Il y a moins de crime entre 1h et 7h du matin
    - Les **années** : Il n'y a pas assez de donnée pour 2015

### 1-1 Dates-Hour / Dates-Minutes

In [ ]:
fig, (axis1,axis2) = plt.subplots(nrows=2, ncols=1, figsize=(18,4)) 
sns.countplot(data=data, x='Dates-hour', ax=axis1)
sns.countplot(data=data, x='Dates-minute', ax=axis2)

In [ ]:
# Dates-hour exploration
data['Dates-hour'].value_counts()[-5:]

In [ ]:
## 

def bin_data_minute(hour):
    if hour >=8 & hour ==0:
        return 'High_hour'
    else:
        return 'Low_hour'
    
data['bin_dates_hour'] = data['Dates-hour'].apply(bin_data_minute)
fig, axis1 = plt.subplots(figsize=(10,20))
sns.countplot(data=data, y='Category', hue='bin_dates_hour',ax=axis1)

##### Conclusion

Ce graphique permet de voir le lien entre le crime et la periode dans laquelle il a été commis.
- Si on se base sur ce graphe on peut voir que la plus part des mesures montre le même modèke, mais certains sont bien différent... 


In [ ]:
data['Dates-minute'].value_counts()[:10]

### Conclusion 

Peut-être qu'un niveau de précision au quart d'heure est suffisant 0/15/30/45, vu que c'est les fréquences les plus fréquentes.

### Adresse

Il y a deux types d'adresse :
- Les adresses contenant un '/'
- Les adresses qui correspondent à un "block"


In [ ]:
# Nombre d'adresse contenant '/'
street_length = len(data[data['Address'].str.contains('/')])
print(street_length)

# Nombre d'adresse en block
print(len(data['Address'])- street_length)

In [ ]:
def bin_address(address):
    if '/' in address and 'of' not in address:
        return 'Street'
    else:
        return 'Block'
data['Address_type'] = data['Address'].apply(bin_address)
data[['Address', 'Address_type']].head(5)

In [ ]:
sns.countplot(data=data, x='Address_type')

In [ ]:
fig, axis1 = plt.subplots(figsize=(10,20))
sns.countplot(data=data, y='Category', hue='Address_type', ax=axis1)

#### Conclusions

- La répartition entre chaque type d'adresse est différente pour chaque type de crime, cette variable est donc pertinente

## 2.1 Adresses récurentes

Certaines adresses apparaissent de nombreuses de fois. On va garder seulement les plus fréquentes et remplacer les autres par 'Others'

In [ ]:
print(len(data[data['Address'] == 'OAK ST / LAGUNA ST']))
print(len(data[data['Address'] == 'LAGUNA ST / OAK ST']))

In [ ]:
crossload = data[data['Address'].str.contains('/')]['Address'].unique()
print('Nombre unique d\'adresse : {0}'.format(len(crossload)))

In [ ]:
topN_address_list = data['Address'].value_counts()
topN_address_list = topN_address_list[topN_address_list >=100]
topN_address_list = topN_address_list.index
print('topN criminal address count is',len(topN_address_list))

In [ ]:
data['Address_clean'] = data['Address']
data.loc[~data['Address'].isin(topN_address_list), "Address_clean"] = 'Others'

data[['Address','Address_clean']].head(5)

In [ ]:
crossload = data[data['Address_clean'].str.contains('/')]
print(crossload.shape)
crossload['Address_clean'].head(3)

In [ ]:
crossload_list = crossload['Address_clean'].unique()
print('Before Adjustment ST_Address length is {0}' .format(len(crossload_list)))

In [ ]:
from tqdm import tqdm

In [ ]:
print(crossload_list[0].split('/')[1].strip() + " / " + crossload_list[0].split('/')[0].strip())
print(crossload_list[0])

In [ ]:
for address in tqdm(crossload_list):
    reverse_address = address.split('/')[1].strip() + " / " + address.split('/')[0].strip()
    data.loc[data['Address_clean'] == reverse_address, 'Address_clean'] = address
crossload_list = data[data['Address_clean'].str.contains('/')]
crossload_list = crossload_list['Address_clean'].unique()
print('Final ST_Address length is {0}' .format(len(crossload_list)))

### 3. PdDistrict

In [ ]:
data[['Category','PdDistrict']].head(3)

In [ ]:
data['PdDistrict'].value_counts()

In [ ]:
sns.countplot(data=data,  x='PdDistrict')